In [1]:
import statsapi as mlb

### Easy to Access Team IDs

In [2]:
teams = ['Angels', 'Orioles', 'Red Sox', 'White Sox', 'Guardians', 'Tigers', 'Royals', 'Twins', 'Yankees', 'Athletics', \
         'Mariners', 'Rays', 'Rangers', 'Blue Jays', 'Diamondbacks', 'Braves', 'Cubs', 'Reds', 'Rockies', 'Marlins', \
         'Astros', 'Dodgers', 'Brewers', 'Nationals', 'Mets', 'Phillies', 'Pirates', 'Cardinals', 'Padres', 'Giants']

team_index_lookup = {}

for team in teams:
    team_index_lookup[team] = mlb.lookup_team(team)[0]['id']

In [3]:
bal_schedule = mlb.schedule(start_date='08/19/2022',end_date='08/21/2022', team = team_index_lookup['Orioles'])
bal_schedule

[{'game_id': 661044,
  'game_datetime': '2022-08-19T23:05:00Z',
  'game_date': '2022-08-19',
  'game_type': 'R',
  'status': 'Pre-Game',
  'away_name': 'Boston Red Sox',
  'home_name': 'Baltimore Orioles',
  'away_id': 111,
  'home_id': 110,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Jordan Lyles',
  'away_probable_pitcher': 'Kutter Crawford',
  'home_pitcher_note': '',
  'away_pitcher_note': '',
  'away_score': '0',
  'home_score': '0',
  'current_inning': '',
  'inning_state': '',
  'venue_id': 2,
  'venue_name': 'Oriole Park at Camden Yards',
  'summary': '2022-08-19 - Boston Red Sox @ Baltimore Orioles (Pre-Game)'},
 {'game_id': 661043,
  'game_datetime': '2022-08-20T20:05:00Z',
  'game_date': '2022-08-20',
  'game_type': 'R',
  'status': 'Scheduled',
  'away_name': 'Boston Red Sox',
  'home_name': 'Baltimore Orioles',
  'away_id': 111,
  'home_id': 110,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Kyle Bradish',
  'away_probable_pitc

In [38]:
print(mlb.standings())

American League East
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   New York Yankees      73  46   -    -      -      -    -  
 2   Tampa Bay Rays        63  54  9.0   36     2      -    -  
 3   Toronto Blue Jays     63  54  9.0   36     3      -    -  
 4   Baltimore Orioles     61  57  11.5  33     5     2.5   43 
 5   Boston Red Sox        59  60  14.0  30     7     5.0   40 

American League Central
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   Cleveland Guardians   63  55   -    -      -      -    -  
 2   Minnesota Twins       61  55  1.0   45     4     1.5   45 
 3   Chicago White Sox     61  58  2.5   42     6     3.0   42 
 4   Kansas City Royals    48  72  16.0  28    10    16.5   28 
 5   Detroit Tigers        45  75  19.0  25    11    19.5   25 

American League West
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   Houston Astros        77  43   -    -      -      -    -  
 2   Seattle Mariners      65  54  1